In [23]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
if not os.environ.get("LANGSMITH_API_KEY"):
  #os.environ["LANGSMITH_API_KEY"] = getpass.getpass()
  os.environ["LANGSMITH_API_KEY"] = ""
if not os.environ.get("OPENAI_API_KEY"):
  # os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")
  os.environ["OPENAI_API_KEY"] = ""


from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [24]:
import langchain
print(langchain.__version__)


0.3.27


In [25]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "data/Code_penal.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

351


In [26]:

# Charger toutes les pages
docs = loader.load()

# Sélectionner une partie (par exemple pages 68 à 100 → index 68 à 99)
extrait_docs = docs[68:100]

# Afficher le texte extrait
for doc in extrait_docs:
    print(doc.page_content)


Partie législative - Livre II : Des crimes et délits contre les personnes - Titre Ier : Des crimes contre l'humanité et contre l'espèce humaine
2° L'extermination ;
3° La réduction en esclavage ;
4° La déportation ou le transfert forcé de population ;
5° L'emprisonnement ou toute autre forme de privation grave de liberté physique en violation des dispositions
fondamentales du droit international ;
6° La torture ;
7° Le viol, la prostitution forcée, la grossesse forcée, la stérilisation forcée ou toute autre forme de violence
sexuelle de gravité comparable ;
8° La persécution de tout groupe ou de toute collectivité identifiable pour des motifs d'ordre politique, racial,
national, ethnique, culturel, religieux ou sexiste ou en fonction d'autres critères universellement reconnus
comme inadmissibles en droit international ;
9° La disparition forcée ;
10° Les actes de ségrégation commis dans le cadre d'un régime institutionnalisé d'oppression systématique et
de domination d'un groupe racial

In [27]:
from PyPDF2 import PdfReader, PdfWriter
import openai
# Charger le fichier PDF
reader = PdfReader("data/Code_penal.pdf")
writer = PdfWriter()

# Vérifier le nombre total de pages
total_pages = len(reader.pages)
print(f"Le PDF contient {total_pages} pages.")

# S’assurer que l'on ne dépasse pas le nombre de pages disponibles
start_page = 67  # page 68
end_page = min(100, total_pages)  # ne pas dépasser les pages existantes

for i in range(start_page, end_page):
    writer.add_page(reader.pages[i])

# Sauvegarder dans un nouveau fichier PDF
with open("data/extrait_docs.pdf", "wb") as f:
    writer.write(f)
print("Extrait PDF sauvegardé dans 'data/extrait_docs.pdf'")


Le PDF contient 351 pages.
Extrait PDF sauvegardé dans 'data/extrait_docs.pdf'


In [28]:
from PyPDF2 import PdfReader
import openai
def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text() + "\n"
    return text


In [29]:
def chunk_text(text, chunk_size=1000, overlap=200):
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunk = text[start:end]
        chunks.append(chunk)
        start += chunk_size - overlap  # chevauchement
    return chunks


In [30]:
openai.api_key = "ta_clef_api_ici"  # mieux : config via variable d'environnement

def get_embedding(text, model="text-embedding-ada-002"):
    response = openai.Embedding.create(input=text, model=model)
    return response['data'][0]['embedding']


In [31]:
import openai
import os

openai.api_key = os.getenv("OPENAI_API_KEY")

if not openai.api_key:
    raise ValueError("OPENAI_API_KEY n'est pas défini dans les variables d'environnement")


In [32]:
def get_embedding(text, model="text-embedding-ada-002"):
    response = openai.embeddings.create(
        model=model,
        input=text
    )
    return response.data[0].embedding




In [57]:
pdf_path = "data/extrait_docs.pdf"
text = extract_text_from_pdf(pdf_path)
chunks = chunk_text(text)
chunks_1 = chunks[0:8]
chunks_2 = chunks[8:15]
embeddings = []

for i, chunk in enumerate(chunks_1):
    emb = get_embedding(chunk)
    embeddings.append(emb)
    print(f"Chunk {i+1}/{len(chunks)} embeded.")

#os.wait()

# for i, chunk in enumerate(chunks_2):
#     emb = get_embedding(chunk)
#     embeddings.append(emb)
#     print(f"Chunk {i+1}/{len(chunks)} embeded.")

# print(f"Total chunks: {len(chunks)}")

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [54]:
chunks_1

["Partie législative - Livre II : Des crimes et délits contre les personnes - Titre Ier : Des crimes contre l'humanité et contre l'espèce humaine\nLivre II : Des crimes et délits contre les personnes\nTitre Ier : Des crimes contre\nl'humanité et contre l'espèce humaine\nSous-titre Ier : Des crimes contre l'humanité\nChapitre Ier : Du génocide\n211-1   Loi n°2004-800 du 6 août 2004 - art. 28 () JORF 7 août 2004   -  Conseil Constit. 2004-498 DC       \n Legif.   \n Plan   \n Jp.Judi.   \n Jp.Admin.   \n Juricaf\nConstitue un génocide le fait, en exécution d'un plan concerté tendant à la destruction totale ou partielle d'un\ngroupe national, ethnique, racial ou religieux, ou d'un groupe déterminé à partir de tout autre critère arbitraire,\nde commettre ou de faire commettre, à l'encontre de membres de ce groupe, l'un des actes suivants :\n– atteinte volontaire à la vie ;\n– atteinte grave à l'intégrité physique ou psychique ;\n– soumission à des conditions d'existence de nature à entraîn